In [ ]:
import pandas as pd
import pysam as ps
import matplotlib.pyplot as plt

In [ ]:
sav_xlsx = pd.read_excel('2021-02-17-AR-HBOC-Variants_for_SAV_project.xlsx')
nonred_exin = pd.read_excel('2021-02-17-AR-nonredundant_intron_exon_junctions_in_387_cancer_disposition-genes.xlsx')
brca_seq = pd.read_excel('BRCA-SEQ_2021-02-07.xlsx')

In [ ]:
naga_vars = pd.read_pickle(r'../../00_StephanData/01_nagaData/01_DataCleaningAnnotating/dfmerge')

In [ ]:
naga_RNAexpr = pd.read_table(r'../../00_StephanData/01_nagaData/hot_warm_variants.RNA_VAF.tsv')

In [ ]:
# for VEP annotations
naga_vars['HGVSg'].drop_duplicates().to_csv('naga_vars_VEP_conf.txt', header=False, index=False)

In [ ]:
# which columns would be duplicated
list(set.intersection(set(naga_vars.columns), set(naga_RNAexpr.columns)))

In [ ]:
naga_RNAexpr = naga_RNAexpr.rename(columns={'PID':'PID_trans'})

In [ ]:
# merge RNA expression to naga table

naga_RNA_merge = pd.merge(naga_vars, naga_RNAexpr[['#CHROM', 'POS', 'REF', 'ALT', 'PID_trans', 'AF_C', 'AF_T', 
                                                   'Alt_T', 'Cov_T', 'Alt_C', 'Cov_C', 'AF_RNA', 
                                                   'AF_RNA_noSkipReads']], 
                          how='left', on=['#CHROM', 'POS', 'REF', 'ALT', 'PID_trans']).drop_duplicates()
naga_RNA_merge.to_pickle('naga_vars_RNA')

up to here important

In [ ]:
# FLOSSIES annotations
#for g in ['BRCA1', 'BRCA2', 'ATM', 'ATR', 'BAP1', 'BARD1', 'BRIP1', 'CDH1', 'CHEK1', 'CHEK2', 'CTNNA1', 'FAM175A',
#          'FANCM', 'GEN1', 'MRE11A', 'NBN', 'PALB2', 'PTEN', 'RAD51B', 'RAD51C', 'RAD51D', 'RECQL', 'RINT1', 
#          'SLX4', 'STK11', 'TP53', 'XRCC2']:
#    print(g) if g in naga_vars['Approved symbol (HGNC)'].unique() else print('No')

In [ ]:
len(sav_xlsx), len(nonred_exin), len(brca_seq), len(naga_vars)

In [ ]:
brca_seq.head()

In [ ]:
brca_seq.columns

In [ ]:
brca_seq['PREDICTION_MUTATIONTASTER'].unique()[:5]

In [ ]:
nonred_exin.head()

In [ ]:
nonred_exin.columns

In [ ]:
sav_xlsx.head()

In [ ]:
sav_xlsx.columns

### Short visualization

In [ ]:
classif = pd.DataFrame(sav_xlsx['KLASSIFIKATION'].value_counts())
classif['classes'] = classif.index
classif.reset_index(drop=True, inplace=True)

In [ ]:
classif['classes'] = pd.Categorical(classif['classes'], categories=['Class 1','Class 2','Class 3 Tendenz 2',
                                                                    'Class 3','Class 3 Tendenz 4','Class 4',
                                                                    'Class 5','pathogen','Poly/neutral','VUS',
                                                                    'keine Angabe'], ordered=True)
classif = classif.sort_values('classes', ignore_index=True)
classif = classif[['classes', 'KLASSIFIKATION']]

In [ ]:
fig = plt.figure()
plt.title('Classification of all variants')
plt.bar(classif['classes'], classif['KLASSIFIKATION'])
plt.xticks(rotation=45, ha='right');

In [ ]:
chrom_uni = pd.DataFrame(sav_xlsx['#CHROM'].value_counts())
chrom_uni['chrom'] = chrom_uni.index
chrom_uni.sort_values(by='chrom', inplace=True)
chrom_uni.reset_index(drop=True, inplace=True)
#.sort()

In [ ]:
fig = plt.figure()
plt.title('Amount of variants per chromosome')
plt.bar(chrom_uni['chrom'], chrom_uni['#CHROM']);

### Combination with Naga data

In [ ]:
naga_vars['#CHROM']=naga_vars['#CHROM'].astype(str)
naga_vars['POS']=naga_vars['POS'].astype(int)
naga_vars['REF']=naga_vars['REF'].astype(str)
naga_vars['ALT']=naga_vars['ALT'].astype(str)

sav_xlsx['#CHROM']=sav_xlsx['#CHROM'].astype(str)
sav_xlsx['POS']=sav_xlsx['POS'].astype(int)
sav_xlsx['REF']=sav_xlsx['REF'].astype(str)
sav_xlsx['ALT']=sav_xlsx['ALT'].astype(str)


merge_naga_hboc = pd.merge(naga_vars, sav_xlsx, how='inner', on=['#CHROM', 'POS', 'REF', 'ALT'])

In [ ]:
len(sav_xlsx), len(naga_vars), len(merge_naga_hboc)

In [ ]:
merge_naga_hboc_nodup_pat = merge_naga_hboc.drop_duplicates(subset=['#CHROM', 'POS', 'REF', 'ALT', 'PID'])
merge_naga_hboc_nodup_var = merge_naga_hboc_nodup_pat.drop_duplicates(subset=['#CHROM', 'POS', 'REF', 'ALT'])
len(merge_naga_hboc_nodup_pat), len(merge_naga_hboc_nodup_var)

In [ ]:
naga_vars_dup = naga_vars.drop_duplicates(subset=['#CHROM', 'POS', 'REF', 'ALT'])
len(naga_vars.drop_duplicates(subset=['#CHROM', 'POS', 'REF', 'ALT']))

In [ ]:
[(t,len(naga_vars_dup[naga_vars_dup['Splice Project Gene Priority']==t])) for t in 
 naga_vars_dup['Splice Project Gene Priority'].unique()]

In [ ]:
[(t,len(naga_vars[naga_vars['Splice Project Gene Priority']==t])) for t in 
 naga_vars['Splice Project Gene Priority'].unique()]

In [ ]:
for_naga = naga_vars[['#CHROM', 'POS', 'REF', 'ALT', 
                      'PID']][naga_vars['Splice Project Gene Priority']!='Cold (387-ACMG-MASTER)'].copy()
for_naga.to_csv('vars_hot_warm_AB.tsv', sep='\t', index=False)

In [ ]:
for_naga['gHGNV']= for_naga['#CHROM'].astype(str)+':'+for_naga['POS'].astype(str)+' '+for_naga['REF']+'>'+for_naga['ALT']

In [ ]:
len(merge_naga_hboc['PID'].unique())

In [ ]:
for i in merge_naga_hboc.columns:
    if 'Splice' in i:
        print(i)

In [ ]:
[(t,len(merge_naga_hboc_nodup_var[merge_naga_hboc_nodup_var['Splice Project Gene Priority']==t])) for t in 
 merge_naga_hboc_nodup_var['Splice Project Gene Priority'].unique()]

In [ ]:
# patients per variant
pat_var_ct = pd.DataFrame(merge_naga_hboc_nodup_pat['HGVSg'].value_counts())
len(pat_var_ct.loc[pat_var_ct['HGVSg']>1])

In [ ]:
naga_pat_var_ct = pd.DataFrame(naga_vars[naga_vars['Splice Project Gene Priority']=='Hot (ACMG / MASTER)'].drop_duplicates(subset=['#CHROM', 'POS', 'REF', 'ALT', 'PID'])['HGVSg'].value_counts())
len(naga_pat_var_ct[(naga_pat_var_ct['HGVSg']>5)])

In [ ]:
naga_hot_5 = naga_vars[naga_vars['HGVSg'].isin(naga_pat_var_ct.index[(naga_pat_var_ct['HGVSg']>5)])]# for i in naga_pat_var_ct[(naga_pat_var_ct['HGVSg']>5)]]

In [ ]:
merge_naga_hboc.columns

In [ ]:
merge_naga_hboc[['HGVSg', 'PosExonRefSeqAccession', 'PosExon_type', 
                 'VEP_Most_Severe_Consequence']][(merge_naga_hboc['PosExonRefSeqAccession']==1)&
                                                 (merge_naga_hboc['VEP_Most_Severe_Consequence']!='missense_variant')]

## Combination with Naga data cleaning

--> not important

In [ ]:
from tqdm import tqdm
from gtfparse import read_gtf

In [ ]:
sav_vars = sav_xlsx.copy()
sav_vars['refseq'] = sav_vars['ID'].str.split(':', expand=True)[0]
sav_vars = sav_vars.drop_duplicates(subset=['#CHROM','POS','ID','REF','ALT'])


In [ ]:
gtf_df=read_gtf('/mnt/g27prist/CMTD/Stephan/bcbio_installation/genomes/Hsapiens/GRCh37/StephanDatabase/GRCH37/annotation/GRCh37_latest_genomic_replace.gtf')
gtf_df['transcript_id']=gtf_df['transcript_id'].str.split('.',expand=True)[0]
gtf_df=gtf_df[gtf_df['transcript_id'].isin(set(sav_vars['refseq']))]

In [ ]:
len(gtf_df['transcript_id'].unique()), len(sav_vars['refseq'].unique())

In [ ]:
def exonPos(trans,pos):
    temp=gtf_df[(gtf_df['transcript_id']==trans)&(gtf_df['seqname'].str.len()<=3)&(gtf_df['feature']=='CDS')]
    if temp.empty:
        print(trans)
        return('err','err')
    strand=temp['strand'].values[0]

    if strand=='+':
        StartEx=temp['start'].min()
        StopEx=temp['end'].max()
    if strand=='-':
        StartEx=temp['end'].max()
        StopEx=temp['start'].min()
    
    
    if strand == '+':
        if pos < StartEx and pos > (StartEx -30):
            dist=StartEx-pos
            return (dist, '5utr_startcodon')
        if pos > StopEx and pos < (StopEx + 30):
            dist=pos-StopEx
            return (dist, '3utr_stopcodon')
        
        OutsideAcceptor=temp[((temp['start']-30)<=pos)&((temp['start'])>pos)]
        InsideAcceptor=temp[((temp['start']+30)>=pos)&((temp['start'])<=pos)]
        
        OutsideDonor=temp[((temp['end']+30)>=pos)&((temp['end'])<pos)]
        InsideDonor=temp[((temp['end']-30)<=pos)&((temp['end'])>=pos)]
        
        if not OutsideAcceptor.empty:
            dist=OutsideAcceptor['start'].values[0]-pos
            return (dist,'outsideAcceptorSite')
        if not InsideAcceptor.empty:
            dist=pos-InsideAcceptor['start'].values[0]
            return (dist,'insideAcceptorSite')
            
        
        if not OutsideDonor.empty:
            dist=pos-OutsideDonor['end'].values[0]
            return (dist,'outsideDonorSite')
            
        if not InsideDonor.empty:
            dist=InsideDonor['end'].values[0]-pos
            return (dist,'insideDonorSite')
    
    
    if strand =='-':
        if pos > StartEx and pos < (StartEx + 30):
            dist=pos-StartEx
            return (dist, '5utr_startcodon')
        if pos < StopEx and pos > (StopEx+30):
            dist=StopEx-pos
            return (dist, '3utr_stopcodon')
        
        OutsideDonor=temp[((temp['start']-30)<=pos)&((temp['start'])>pos)]
        InsideDonor=temp[((temp['start']+30)>=pos)&((temp['start'])<=pos)]
        
        OutsideAcceptor=temp[((temp['end']+30)>=pos)&((temp['end'])<pos)]
        InsideAcceptor=temp[((temp['end']-30)<=pos)&((temp['end'])>=pos)]
        
        if not OutsideAcceptor.empty:
            dist=pos-OutsideAcceptor['end'].values[0]
            return (dist,'outsideAcceptorSite')
        if not InsideAcceptor.empty:
            dist=InsideAcceptor['end'].values[0]-pos
            return (dist,'insideAcceptorSite')
            
        
        if not OutsideDonor.empty:
            dist=OutsideDonor['start'].values[0]-pos
            return (dist,'outsideDonor')
            
        if not InsideDonor.empty:
            dist=pos-InsideDonor['start'].values[0]
            return (dist,'insideDonor')
    
    return ('01','01')

In [ ]:
posRelativeRefSeq = []
Utr_cod=[]

for pos,trans in tqdm(zip(sav_vars["POS"],sav_vars["refseq"]),total=len(sav_vars)):
    ex_pos=exonPos(trans,int(pos))
    posRelativeRefSeq.append(ex_pos[0])
    Utr_cod.append(ex_pos[1])
#df_merge['PosExonRefSeqAccession']=posRelativeRefSeq

sav_vars['PosExonRefSeqAccession']=posRelativeRefSeq
sav_vars['PosExon_type']=Utr_cod

sav_vars=sav_vars[sav_vars['PosExon_type']!='01']

In [ ]:
sav_vars.to_pickle('sav_vars_PosExon')

## Visualization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', None)


In [ ]:
# set style of figures
plt.style.use('bmh')
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 20
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)

In [ ]:
# integrate merged df
#sav_vars=pd.read_pickle('sav_vars_PosExon')
#sav_vars['max_gnomAD_AF']=sav_vars['max_gnomAD_AF'].replace('.',np.nan).astype('float')
sav_vars['Splice Project Gene Priority'] = 'Hot (ACMG / MASTER)'

In [ ]:
# plot exon and region where variants are (outside/inside and acceptor/donor)
def plotEx(ax,pos=None,Type=None):
    ax.arrow(0, 8, 5, 0, head_width=1, head_length=0.4, fc='b', ec='k',linewidth=2)
    ax.plot((0,10),(4,4), linewidth=5,color='black')
    #ax.plot((3,6),(4,4), linewidth=10,color='blue')
    ax.axis('off')
    ax.text(5,4,'EXON', color='white',fontdict={'size':16},ha='center', va='center')
    ax.add_patch(patches.Rectangle((3, 2), 4, 4, linewidth=1, edgecolor='b', facecolor='b', alpha=1, zorder=3))
    if pos == 'outside' and Type=='donor':
        ax.add_patch(patches.Rectangle((7, 1), 1, 6, linewidth=3, edgecolor='black', facecolor='r', alpha=0.2, zorder=3))
        ax.add_patch(patches.Rectangle((7, 1), 1, 6, linewidth=1, edgecolor='black', facecolor='none', alpha=1, zorder=3))
        
    if pos == 'inside' and Type =='donor':
        ax.add_patch(patches.Rectangle((6, 1), 1, 6, linewidth=3, edgecolor='black', facecolor='r', alpha=0.2, zorder=3))
        ax.add_patch(patches.Rectangle((6, 1), 1, 6, linewidth=1, edgecolor='black', alpha=1,facecolor='none', zorder=3))
        
    if pos == 'inside' and Type=='acceptor':
        ax.add_patch(patches.Rectangle((3, 1), 1, 6, linewidth=3, edgecolor='black', facecolor='r', alpha=0.2, zorder=3))
        ax.add_patch(patches.Rectangle((3, 1), 1, 6, linewidth=1, edgecolor='black', facecolor='none', alpha=1, zorder=3))
        
    if pos == 'outside' and Type =='acceptor':
        ax.add_patch(patches.Rectangle((2, 1), 1, 6, linewidth=3, edgecolor='black', facecolor='r', alpha=0.2, zorder=3))
        ax.add_patch(patches.Rectangle((2, 1), 1, 6, linewidth=1, edgecolor='black', alpha=1,facecolor='none', zorder=3))    
        
    ax.set(ylim=(0,10))

In [ ]:
# similar to DistanceDist(), but improved (df as attribute, x = range() not negative)
# PosExonRefSeqAccession = distance from exon-intron boundary
# count all variants with same distance from exon-intron boundary
def PlotAccDon(df,ax,key=None,value=None, col='grey'):
    if key==None:
        v=dict(df['PosExonRefSeqAccession'].value_counts())
        x=list(range(0,26))
        y=[v[x] if x in v else 0 for x in x]
        
        ax.bar(x,y, color=col)
        ax.set(title='position of variants - all gene-baskets',
               xlabel='position relative to Intron-Exon boundary',ylabel='number of variants')        
    else:
        df_temp=df[df[key]==value]
        v=dict(df_temp['PosExonRefSeqAccession'].value_counts())
        x=list(range(0,26))
        y=[v[x] if x in v else 0 for x in x]
        ax.bar(x,y, color=col)
        ax.set(title='position of variants - {}'.format(value),
               xlabel='position relative to Intron-Exon boundary', ylabel='number of variants') 

In [ ]:
# horizontal bar plot with value counts for different consequences of variants
colDict={}
for x,y in zip(sns.color_palette(n_colors=len(sav_vars['KONSEQUENZ'].unique())),
               list(dict(sav_vars['KONSEQUENZ'].value_counts()))):
    colDict[y]=x
def StackedPlot(df,ax,key=None,value=None):
    left=0
    l=[]
    df_temp=df[df[key]==value]
    
    for k,v in df_temp['KONSEQUENZ'].value_counts().items():
        p=ax.barh(1,v,4,left=left,label=k,color=colDict[k])
        left=left+v
    ax.set_yticklabels([''])
    for tic in ax.yaxis.get_major_ticks():
                tic.tick1line.set_visible(False)
    ax.legend(loc='lower left',bbox_to_anchor=(1,0),ncol=1)

In [ ]:
# brca1/2 only hot splice project priority --> 
# combine previous plots to subplots
with PdfPages('allSAVs_Andreas.pdf') as pdf:
    #for temperature in ['Hot (ACMG / MASTER)','Warm (MASTER-ACMG)','Cold (387-ACMG-MASTER)']:
    df=sav_vars#[df_merge_brca12['Splice Project Gene Priority']=='Hot (ACMG / MASTER)']        
    fig,ax=plt.subplots(4,3,figsize=(20,20))
    plt.suptitle('Hot (ACMG / MASTER)')
        
    v=plotEx(ax[0,0],pos='outside', Type='acceptor')
    PlotAccDon(df,ax[0,1],key='PosExon_type',value='outsideAcceptorSite')
    StackedPlot(df,ax[0,2],key='PosExon_type',value='outsideAcceptorSite')        
        
    v=plotEx(ax[1,0],pos='inside', Type='acceptor')
    PlotAccDon(df,ax[1,1],key='PosExon_type',value='insideAcceptorSite')
    StackedPlot(df,ax[1,2],key='PosExon_type',value='insideAcceptorSite')        
        
    v=plotEx(ax[2,0],pos='outside', Type='donor')
    PlotAccDon(df,ax[2,1],key='PosExon_type',value='outsideDonorSite')
    StackedPlot(df,ax[2,2],key='PosExon_type',value='outsideDonorSite')        
        
    v=plotEx(ax[3,0],pos='inside', Type='donor')
    PlotAccDon(df,ax[3,1],key='PosExon_type',value='insideDonor')
    StackedPlot(df,ax[3,2],key='PosExon_type',value='insideDonor')        
    
    plt.tight_layout()
    plt.subplots_adjust(top=.9)
    pdf.savefig()
    plt.close()